In [37]:
import os
import numpy as np
import torch
import torch.nn as nn
import chromadb
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image
from pathlib import Path


# Faceent model defined

In [38]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.facenet=InceptionResnetV1(pretrained='casia-webface' ,classify=False)
    self.facenet.last_bn=torch.nn.Identity()
    self.facenet.logits=torch.nn.Identity()

  def forward(self,x):
    return self.facenet(x)
  

In [39]:
# Path to ChromaDB storage
CHROMA_DB_PATH = "../../../data/embeddings"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [40]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load your fine-tuned FaceNet model
state_dict_path = Path(r"models\facenet\fine_tuned\fine_tuned_model.pth")
state_dict = torch.load(state_dict_path, weights_only=False, map_location=device)

facenet_model = Model()  # Initialize your custom model
facenet_model.load_state_dict(state_dict)  # Load the fine-tuned weights
facenet_model.to(device).eval()  # Move model to device and set to evaluation mode


Model(
  (facenet): InceptionResnetV1(
    (conv2d_1a): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2a): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2b): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_3b): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_

In [ ]:
# Initialize ChromaDB client with persistent storage
chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH)

# Create or get existing collection
collection = chroma_client.get_or_create_collection(name="face_embeddings")

In [45]:
# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the tensor

])

In [46]:
def preprocess_image(image_path):
    """Load and preprocess an image for FaceNet."""
    try:
        with Image.open(image_path).convert('RGB') as img:
            img = transform(img).unsqueeze(0).to(device)
        return img
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

In [47]:
def get_embedding(image_path):
    """Generate a FaceNet embedding for an image."""
    img = preprocess_image(image_path)
    if img is None:
        return None
    with torch.no_grad():
        embedding = facenet_model(img)
    return embedding.cpu().numpy().flatten().tolist()  # Convert to list for ChromaDB


# store embedding main

In [48]:
def store_embeddings(image_folder, source_type="cctv"):
    """
    Extract embeddings and store them in ChromaDB with source metadata, avoiding duplicates.
    """
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    # Retrieve all existing entries in the collection to check for duplicates
    existing_entries = collection.get()
    existing_ids = existing_entries['ids'] if existing_entries else []

    # If the database is empty, existing_ids will be an empty list
    print(f"Existing IDs in the database: {existing_ids}")
    
    for img_name in image_files:
        img_path = os.path.join(image_folder, img_name)
        emb = get_embedding(img_path)
        
        if emb is not None:
            print(source_type)
            print(f"Embedding for {img_name}: {emb[:5]}...")  # Print first 5 values of the embedding
            print(f"Shape of embedding for {img_name}: {np.shape(emb)}")
            
            if img_name not in existing_ids:
                # Add the embedding to the collection
                collection.add(
                    ids=[img_name],
                    embeddings=[emb],
                    metadatas=[{"source": source_type, "filename": img_name}]
                )
                print(f"Stored {img_name} from {source_type} in ChromaDB.")
            else:
                print(f"Embedding for {img_name} already exists, skipping storage.")
        else:
            print(f"Failed to extract embedding for {img_name}, skipping.")




In [49]:
# Store Bizos (anchor) images as permanent
bizos_folder = "../../../data/raw/bizos_export"
store_embeddings(bizos_folder, source_type="bizos")

Existing IDs in the database: []
bizos
Embedding for aayush.jpg: [0.03815271705389023, 0.06064104288816452, 0.022444400936365128, -0.016721675172448158, -0.01605788990855217]...
Shape of embedding for aayush.jpg: (512,)
Stored aayush.jpg from bizos in ChromaDB.
bizos
Embedding for abhishek.jpg: [-0.022191287949681282, 0.1365884244441986, 0.06177190691232681, -0.015215196646749973, -0.03681989759206772]...
Shape of embedding for abhishek.jpg: (512,)
Stored abhishek.jpg from bizos in ChromaDB.
bizos
Embedding for abiral.jpg: [0.06698190420866013, -0.07517854124307632, -0.04244042932987213, -0.010340902954339981, -0.04142653942108154]...
Shape of embedding for abiral.jpg: (512,)
Stored abiral.jpg from bizos in ChromaDB.
bizos
Embedding for amrit.jpg: [-0.05865606293082237, 0.07505728304386139, 0.029383836314082146, 0.006725209299474955, 0.08892461657524109]...
Shape of embedding for amrit.jpg: (512,)
Stored amrit.jpg from bizos in ChromaDB.
bizos
Embedding for arbin.jpg: [-0.0192382838577

Add of existing embedding ID: jeshika.jpg


bizos
Embedding for jeshika.jpg: [0.004057579208165407, 0.02647269330918789, 0.08410027623176575, -0.008282909169793129, -0.016753215342760086]...
Shape of embedding for jeshika.jpg: (512,)
Stored jeshika.jpg from bizos in ChromaDB.


Add of existing embedding ID: krish.jpg


bizos
Embedding for krish.jpg: [-0.030208788812160492, -0.009629013948142529, 0.04593949019908905, -0.018833912909030914, -0.04059198126196861]...
Shape of embedding for krish.jpg: (512,)
Stored krish.jpg from bizos in ChromaDB.


...

bizos
Embedding for sushmita.jpg: [0.05839676782488823, -0.053365956991910934, -0.052408698946237564, 0.02538725547492504, -0.04192688688635826]...
Shape of embedding for sushmita.jpg: (512,)
Stored sushmita.jpg from bizos in ChromaDB.


Add of existing embedding ID: wonika.jpg


bizos
Embedding for wonika.jpg: [-0.02487356960773468, -0.04245855286717415, 0.013099724426865578, -0.014592994004487991, 0.0008573823142796755]...
Shape of embedding for wonika.jpg: (512,)
Stored wonika.jpg from bizos in ChromaDB.


Add of existing embedding ID: yubraj.jpg


bizos
Embedding for yubraj.jpg: [-0.018091915175318718, -0.07017626613378525, -0.04669570550322533, 0.02548450417816639, 0.048064131289720535]...
Shape of embedding for yubraj.jpg: (512,)
Stored yubraj.jpg from bizos in ChromaDB.


In [51]:
raise ValueError("Created Embeddings")

ValueError: Created Embeddings

# Deleting the chromadb 

In [36]:
# Retrieve all embeddings' IDs from the collection
results = collection.get()  # Get all stored embeddings
existing_ids = results['ids']  # Extract the IDs from the response

# Delete all embeddings by their IDs
collection.delete(ids=existing_ids)

print("Cleared the ChromaDB collection.")


Cleared the ChromaDB collection.


# Check number of collection in the databse

In [19]:
# Get all embeddings in the collection after deletion
results = collection.get()

# Check if the collection is empty
if not results['ids']:
    print("The ChromaDB collection is empty.")
else:
    print(f"There are still {len(results['ids'])} embeddings in the collection.")


The ChromaDB collection is empty.


In [138]:
print(f"Total entries in collection after storing: {collection.count()}")


Total entries in collection after storing: 11


# check metadata

In [35]:
results = collection.get()
print(f"Results structure: {results}")  
print(f"IDs: {results['ids']}")
print(f"Metadata: {results['metadatas']}")


Results structure: {'ids': ['Aayush Dangol.jpg', 'aayush.jpg', 'abhishek.jpg', 'abiral.jpg', 'amrit.jpg', 'arbin.jpg', 'bitisha.jpg', 'cimrika.jpg', 'david.jpg', 'human.jpg', 'jeshika.jpg', 'krish.jpg', 'namuna.jpg', 'nischal.jpg', 'nitin.jpg', 'prashant.jpg', 'rakesh.jpg', 'rejina.jpg', 'rezi.jpg', 'Ronak Shrestha.jpg', 'ronak.jpg', 'roshan.jpg', 'safal.jpg', 'sapana.jpg', 'Sardul Ojha.jpg', 'sardul.jpg', 'sashank.jpg', 'sumit.jpg', 'surab.jpg', 'sushmita.jpg', 'wonika.jpg', 'yubraj.jpg'], 'embeddings': None, 'documents': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'uris': None, 'data': None, 'metadatas': [{'filename': 'Aayush Dangol.jpg', 'source': 'bizos'}, {'filename': 'aayush.jpg', 'source': 'bizos'}, {'filename': 'abhishek.jpg', 'source': 'bizos'}, {'filename': 'abiral.jpg', 'source': 'bizos'}, {'filename': 'amrit.jpg', 'source': 'b

# confirm the embedding of an id

In [115]:
retrieved_sardul = collection.get(ids=["Sardul Ojha.jpg"], include=["embeddings", "metadatas"])
print(f"Retrieved Sardul's data: {retrieved_sardul}")


Retrieved Sardul's data: {'ids': ['Sardul Ojha.jpg'], 'embeddings': array([[ 3.34398705e-04,  2.10383181e-02, -4.86497395e-03,
         1.61283109e-02,  9.89158079e-03, -4.38145176e-02,
        -1.97479669e-02,  1.69756170e-02,  4.68918420e-02,
         5.25346361e-02, -3.06827133e-03,  8.53475183e-03,
         1.72120444e-02,  3.82984942e-03,  2.99394839e-02,
        -4.06905115e-02,  3.38513628e-02,  1.93358362e-02,
         1.12689631e-02,  9.56705064e-02, -3.20110656e-02,
         3.32885049e-02, -5.53778075e-02,  3.00541025e-04,
        -2.47600600e-02, -1.92050100e-03,  2.02486571e-02,
         1.37238670e-02, -4.86466102e-02, -2.22689789e-02,
         2.22953092e-02,  3.21224006e-03,  5.51130064e-02,
         1.73916612e-02,  3.00018899e-02,  5.51520884e-02,
        -3.59129459e-02,  1.16911456e-02,  2.16147210e-02,
        -1.18404098e-01, -4.95384783e-02, -7.80066997e-02,
         1.06441081e-01, -1.97754707e-02, -1.66220702e-02,
         1.27440849e-02,  1.71025060e-02, -1.55

In [114]:
retrieved = collection.get(include=['metadatas', 'embeddings'])
print(f"Retrieved data: {retrieved}")


Retrieved data: {'ids': ['Sardul Ojha.jpg', 'Krish.jpg', 'Ronak Shrestha.jpg', 'Yubraj Maharjan.jpg', 'Amrit Regmi.jpg', 'Cimrika Bajracharya.jpg', 'Aayush Dangol.jpg', 'David Bista.jpg', 'Sumit Prasad Sah.jpg', 'Rejina Sharma.jpg', 'test_id'], 'embeddings': array([[ 0.0003344 ,  0.02103832, -0.00486497, ..., -0.00160347,
        -0.0078511 , -0.05077709],
       [-0.00943508, -0.07421143, -0.03008049, ...,  0.04189048,
         0.05244018, -0.02205809],
       [-0.01683637, -0.00752066,  0.01524568, ...,  0.00072198,
        -0.0488276 , -0.0598639 ],
       ...,
       [ 0.01392234, -0.04653224,  0.01280447, ..., -0.03164221,
        -0.0190408 , -0.02550483],
       [ 0.0375364 ,  0.02359789, -0.00650262, ..., -0.03225034,
        -0.03145462,  0.02834947],
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ]]), 'documents': None, 'uris': None, 'data': None, 'metadatas': [{'filename': 'Sardul Ojha.jpg', 'source': 'bizos'}, {'filename': '